### 1. Library 반입

In [1]:
import pandas as pd
from scipy import stats 
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

### 2. Data 반입
#### Importing Pre-DCS data

In [14]:
names = ['Cell', 'Baseline', 'NumberOfSpike', 'MeanFiringRate', 'TrainDuration', 'PB-AHP', 'APOnsetTime', 'InjectedCurrent']
names

['Cell',
 'Baseline',
 'NumberOfSpike',
 'MeanFiringRate',
 'TrainDuration',
 'PB-AHP',
 'APOnsetTime',
 'InjectedCurrent']

In [21]:
df_pre = pd.read_csv('D:/DataAtCSBD/2022/2_DCS_MEF_effect/DCS_pair/Baseline/Depol.txt', sep="\t")
df_pre.columns = names

(                 Cell   Baseline  NumberOfSpike  MeanFiringRate  \
 0  220228_B6_PN_#01_B -70.485750            0.0             0.0   
 1  220228_B6_PN_#02_B -68.434251            0.0             0.0   
 2  220303_B6_PN_#02_B -72.334708            0.0             0.0   
 3  220307_B6_PN_#03_B -69.615530            0.0             0.0   
 4  220307_B6_PN_#04_B -69.637643            0.0             0.0   
 
    TrainDuration    PB-AHP  APOnsetTime InjectedCurrent  
 0            0.0  0.498137          0.0       Level_000  
 1            0.0  1.176345          0.0       Level_000  
 2            0.0  1.487313          0.0       Level_000  
 3            0.0 -0.340628          0.0       Level_000  
 4            0.0 -0.698434          0.0       Level_000  ,
 (120, 8))

In [30]:
protocol = ['pre'] * 120
df_protocol = pd.DataFrame({'protocol': protocol})
df_pre = pd.concat([df_pre, df_protocol], axis=1)
df_pre.head()

,Cell,Baseline,NumberOfSpike,MeanFiringRate,TrainDuration,PB-AHP,APOnsetTime,InjectedCurrent,protocol
0,220228_B6_PN_#01_B,-70.485750,0.0,0.0,0.0,0.498137,0.0,Level_000,pre
1,220228_B6_PN_#02_B,-68.434251,0.0,0.0,0.0,1.176345,0.0,Level_000,pre
2,220303_B6_PN_#02_B,-72.334708,0.0,0.0,0.0,1.487313,0.0,Level_000,pre
3,220307_B6_PN_#03_B,-69.615530,0.0,0.0,0.0,-0.340628,0.0,Level_000,pre
4,220307_B6_PN_#04_B,-69.637643,0.0,0.0,0.0,-0.698434,0.0,Level_000,pre


#### Importing Post-DCS data

In [22]:
df_post = pd.read_csv("D:/DataAtCSBD/2022/2_DCS_MEF_effect/DCS_pair/DCS/Depol.txt", sep='\t', header=0, names=names)

(                 Cell   Baseline  NumberOfSpike  MeanFiringRate  \
 0  220228_B6_PN_#01_D -69.741295            0.0             0.0   
 1  220228_B6_PN_#02_D -69.827715            0.0             0.0   
 2  220303_B6_PN_#02_D -71.726745            0.0             0.0   
 3  220307_B6_PN_#03_D -69.054920            0.0             0.0   
 4  220307_B6_PN_#04_D -71.923097            0.0             0.0   
 
    TrainDuration    PB-AHP  APOnsetTime InjectedCurrent  
 0            0.0 -0.100182          0.0       Level_000  
 1            0.0  1.552900          0.0       Level_000  
 2            0.0  1.545960          0.0       Level_000  
 3            0.0  0.525158          0.0       Level_000  
 4            0.0  0.434081          0.0       Level_000  ,
 (120, 8))

In [29]:
protocol = ['post'] * 120
df_protocol = pd.DataFrame({'protocol': protocol})
df_post = pd.concat([df_post, df_protocol], axis=1)
df_post.head()

,Cell,Baseline,NumberOfSpike,MeanFiringRate,TrainDuration,PB-AHP,APOnsetTime,InjectedCurrent,protocol
0,220228_B6_PN_#01_D,-69.741295,0.0,0.0,0.0,-0.100182,0.0,Level_000,post
1,220228_B6_PN_#02_D,-69.827715,0.0,0.0,0.0,1.552900,0.0,Level_000,post
2,220303_B6_PN_#02_D,-71.726745,0.0,0.0,0.0,1.545960,0.0,Level_000,post
3,220307_B6_PN_#03_D,-69.054920,0.0,0.0,0.0,0.525158,0.0,Level_000,post
4,220307_B6_PN_#04_D,-71.923097,0.0,0.0,0.0,0.434081,0.0,Level_000,post


In [31]:
df_post.Cell.unique()

array(['220228_B6_PN_#01_D', '220228_B6_PN_#02_D', '220303_B6_PN_#02_D',
       '220307_B6_PN_#03_D', '220307_B6_PN_#04_D', '220311_B6_PN_#01_D',
       '220311_B6_PN_#02_D', '220311_B6_PN_#04_D', '220321_B6_PN_#02_D'],
      dtype=object)

### 3. 정규성 검정
#### 3.1. Group data 분할

In [36]:
df_M_WT=df[(df['Sex']=='Male')&(df['Genotype']=='WT')]
df_M_HM=df[(df['Sex']=='Male')&(df['Genotype']=='HM')]
df_F_WT=df[(df['Sex']=='Female')&(df['Genotype']=='WT')]
df_F_HM=df[(df['Sex']=='Female')&(df['Genotype']=='HM')]

#### 3.2. Amplitude data에 대한 정규성 검정

In [37]:
stats.shapiro(df_M_WT['Amplitude']),stats.shapiro(df_M_HM['Amplitude']),stats.shapiro(df_F_WT['Amplitude']),stats.shapiro(df_F_HM['Amplitude'])

(ShapiroResult(statistic=0.8285818099975586, pvalue=0.0051416330970823765),
 ShapiroResult(statistic=0.9588784575462341, pvalue=0.580128014087677),
 ShapiroResult(statistic=0.9640830159187317, pvalue=0.736049473285675),
 ShapiroResult(statistic=0.9430546760559082, pvalue=0.20891238749027252))

#### 3.3. Frequency data에 대한 정규성 검정

In [38]:
stats.shapiro(df_M_WT['Frequency']),stats.shapiro(df_M_HM['Frequency']),stats.shapiro(df_F_WT['Frequency']),stats.shapiro(df_F_HM['Frequency'])

(ShapiroResult(statistic=0.6981892585754395, pvalue=0.00011027670552721247),
 ShapiroResult(statistic=0.9219226837158203, pvalue=0.13981656730175018),
 ShapiroResult(statistic=0.9372907280921936, pvalue=0.3171420693397522),
 ShapiroResult(statistic=0.787955105304718, pvalue=0.00024735438637435436))

### 4. 등분산 검정
#### 4.1. Amplitude data에 대한 등분산 검정

In [39]:
stats.bartlett(df_M_WT['Amplitude'], 
               df_M_HM['Amplitude'], 
               df_F_WT['Amplitude'], 
               df_F_HM['Amplitude'])

BartlettResult(statistic=11.037121569084254, pvalue=0.011526829418126772)

#### 4.2. Frequency data에 대한 등분산 검정

In [40]:
stats.bartlett(df_M_WT['Frequency'], 
               df_M_HM['Frequency'], 
               df_F_WT['Frequency'], 
               df_F_HM['Frequency'])

BartlettResult(statistic=10.510591849823903, pvalue=0.01468921839673509)

### 5. 1-way ANOVA 
#### 5.1. Amplitude data에 대한 1-way ANOVA

In [41]:
stats.f_oneway(df_M_WT['Amplitude'], 
               df_M_HM['Amplitude'],
               df_F_WT['Amplitude'], 
               df_F_HM['Amplitude'])

F_onewayResult(statistic=0.7643569847605557, pvalue=0.5178499613570902)

#### 5.2. Frequency data에 대한 1-way ANOVA

In [42]:
stats.f_oneway(df_M_WT['Frequency'], 
               df_M_HM['Frequency'],
               df_F_WT['Frequency'], 
               df_F_HM['Frequency'])

F_onewayResult(statistic=5.731497108024772, pvalue=0.0014475563821714067)

### 6. 2-way ANOVA
#### 6.1. Amplitude data에 대한 2-way ANOVA

In [43]:
model = ols('Amplitude~C(Sex)+C(Genotype)+C(Sex):C(Genotype)', df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(Sex),1.0,121.344431,121.344431,0.541563,0.464244
C(Genotype),1.0,391.928380,391.928380,1.749187,0.190284
C(Sex):C(Genotype),1.0,0.520044,0.520044,0.002321,0.961713
Residual,70.0,15684.425183,224.063217,NaN,NaN


#### 6.2. Frequency data에 대한 2-way ANOVA

In [44]:
model2 = ols('Frequency~C(Sex)+C(Genotype)+C(Sex):C(Genotype)', df).fit()
anova_lm(model2)

,df,sum_sq,mean_sq,F,PR(>F)
C(Sex),1.0,168.723674,168.723674,14.102035,0.000355
C(Genotype),1.0,6.657070,6.657070,0.556402,0.458211
C(Sex):C(Genotype),1.0,30.342594,30.342594,2.536054,0.115779
Residual,70.0,837.514377,11.964491,NaN,NaN
